## Multiparty XGBoost with Federated Training
We will now discuss running XGBoost in the federated setting. Unlike the previous exercise, in the federated setting all data stays on its respective machine. This eliminates the need to transfer over the network which incurs high overhead and requires significant bandwidth. Instead, in the federated setting in each iteration each party sends a summary of the update made to its model. The aggregator then aggregates these updates, applies the aggregated update to its model, and broadcasts the new model to all parties. The parties then train locally with the new model and sends the update to the aggregator.

![title](img/exercise3.png)

In our project, all this is abstracted away. The central server simply starts the training, and everything else is performed automatically.

Import some helper functions.

In [1]:
import pandas as pd
import subprocess
from Utils import start_job

### Edit hosts.config (central server)
The `hosts.config` file should contain the IPs and ports of all workers in the federation. 
Retrieve the IPs of all members in the federation from the PKI and write it to the hosts.config file.

**Only the central server has to do this step.**

In [ ]:
# Get the IPs of all members in your federation and add it to hosts.config
import importlib
import Federation
importlib.reload(Federation)

members = ["chester", "rishabh", "wenting"]
pki = Federation.PKI()
with open("hosts.config", "w+") as hosts:
    for member in members:
        IP, key = pki.lookup(member)
        
        # Write the member's IP address and port 5522 to hosts.config
        hosts.write(IP +":5522\n")
    

### Set Variables For Network Analysis
We'll walk you through inspecting packets during this tutorial as well to make sure that the network topology is indeed federated. For each variable below, fill in the corresponding IP (don't worry about the ordering of the worker nodes).

In [4]:
master = '0'
worker_1 = '1'
worker_2 = '2'
worker_3 = '3'

### Modifying the Training Script (central server)
We will now modify the script that will be run for federated training. Load it in by running the following cell. The contents of the script should appear in the cell. 

The central server controls the training. If you're the central server, you can play with the `params` argument passed into the `train()` function. A list of possible parameters and their descriptions can be found [here](https://xgboost.readthedocs.io/en/latest/parameter.html).

**Only the central server has to do this step.**

In [ ]:
%load train_model.py

### Using tcpdump to Capture Packets
We will be using `tcpdump` to monitor the network traffic during training. The cell below spawns a process that records all incoming network traffic.

In [ ]:
tcpdump_cmd = 'tcpdump -ni en0 -s0 -w capture.pcap'
tcpdump_process = subprocess.Popen(tcpdump_cmd, stdout=subprocess.PIPE, shell=True)

### Start Job (central server)
After modifying the script, we can start our job! We can use the `start_job()` helper function to do so.
`start_job(num_parties, memory, script_path)` takes in three parameters:
* num_parties: The number of parties in the federation. This should be the same as the number of IPs added to hosts.config
* memory: The amount of memory to use for this job on each party's machine
* script_path: The absolute path to the script we want to run

**Only run this cell if you're the central server.**

In [ ]:
start_job(2, 3, "/home/$USER/train_model.py")

Kill the tcpdump process once training has finished as we no longer need to monitor network traffic

In [ ]:
tcpdump_process.terminate()

## Network Analysis
In the federated setting, parties don't communicate with each other -- they only communicate with the aggregator. We'll monitor network traffic in this section to show this isolated communication, and also to show that the communication of updates (as in the federated setting) requires less bandwidth than the transfer of whole raw datasets (as in the centralized training scenario from Exercise 2). 

First, let's convert the `pcap` file we created with `tcpdump` to a `.csv`:

In [ ]:
!tshark -r capture.pcap -T fields -e frame.number -e eth.src -e eth.dst -e ip.src -e ip.dst -e frame.len -E header=y -E separator=, > capture.csv

Load in the `.csv` created by tshark into a pandas DataFrame and drop all rows that have `NaN` in either of the IP columns. Then, we rename the IPs corresponding to the members of the federation for easier reading.

In [2]:
capture = pd.read_csv('capture.csv', names=['Frame Number', 'Ethernet Source', 'Ethernet Destination', 
                                            'IP Source', 'IP Destination', 'Frame Length'], header=0)
capture.dropna(subset=['IP Source', 'IP Destination'], inplace=True)

labels = {master: 'Master', worker_1: 'worker_1', worker_2: 'worker_2', worker_3: 'worker_3'}
capture.replace(labels, inplace=True)

### Preprocessing
Create a new column in the table to see the communications to and from this particular node. In order to get the total number of bytes transmitted for each type of transmission, group by this new column and then sum all the frame lengths. In order to get the total number of packets sent by each transmission, simply count how many times that particular transmission occurs.

In [6]:
capture['Transmission'] = capture.apply(lambda row: row['IP Source'] + ' -> ' + row['IP Destination'], axis=1)
count_bytes = capture.groupby('Transmission', as_index=False)['Transmission', 'Frame Length'].sum()
count_bytes.rename(mapper={'Frame Length': 'Total Bytes Transmitted'}, inplace=True, axis=1)
count_packets = capture['Transmission'].value_counts().rename_axis('Transmission').reset_index(name='Number of Packets')

### Results
In the cell below, we perform a join on the two tables, sort by the total number of bytes transmitted, and show the total communcations

In [7]:
count_bytes.set_index('Transmission', inplace=True)
count_packets.set_index('Transmission', inplace=True)
counts = count_packets.join(count_bytes, on='Transmission')
counts.sort_values(by='Total Bytes Transmitted', inplace=True, ascending=False)
counts

,Number of Packets,Total Bytes Transmitted
Transmission,,
172.217.5.102 -> 192.168.42.5,819,1155696
151.101.189.140 -> 192.168.42.5,451,570386
172.217.0.34 -> 192.168.42.5,151,178864
172.217.5.97 -> 192.168.42.5,92,127184
23.203.221.142 -> 192.168.42.5,207,104438
192.168.42.5 -> 23.203.221.142,230,51367
192.168.42.5 -> 151.101.189.140,366,44163
192.168.42.5 -> 172.217.5.102,505,40062
192.168.42.5 -> 216.58.194.162,66,25723


## Modifying the Evaluation Script
We'll now use the model we trained in the previous step to make predictions on our test data. Load in the test script like in the previous step. 

**Again, only the central server has to do these steps.**

In [ ]:
%load test_model.py

In [ ]:
start_job(2, 3, "/home/$USER/test_model.py")